In [3]:

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import backtrader.feeds as btfeed

# Import pyfolio
import pyfolio as pf


# Create a Stratey
class TestStrategy(bt.Strategy):
    
    params = (('size', 100000),)

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.count = 0
        self.buybars = [1, 150]
        self.closebars = [100, 250]
        
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        
         # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        # Add a Stochastic indicator
        self.stoch = bt.indicators.StochasticSlow(self.datas[0])
        
        # Indicators for the plotting show
#         self.stoc = bt.talib.STOCH(self.data.high, self.data.low, self.data.close,
#                                    fastk_period=14, slowk_period=3, slowd_period=3)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        
        if order.status == order.Completed:
            ep = order.executed.price
            es = order.executed.size
            tv = ep * es
            leverage = self.broker.getcommissioninfo(self.data).get_leverage()
            self.margin = abs((ep * es) / leverage)

            if 'name' in order.info:
                if order.info['name'] == 'Entry':
                    print('{}: Entry Order Completed '.format(self.dt))
                    print('{}: Order Executed Price: {}, Executed Size {}'.format(self.dt,ep,es,))
                    print('{}: Position Value: {} Margin Used: {}'.format(self.dt,tv,self.margin))
                elif order.info['name'] == 'MCO':
                    print('{}: WARNING: Margin Close Out'.format(self.dt))
                else:
                    print('{}: Close Order Completed'.format(self.dt))

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            print('{}: Trade closed '.format(self.dt))
            print('{}: PnL Gross {}, Net {}\n\n'.format(self.dt,
                                                round(trade.pnl,2),
                                                round(trade.pnlcomm,2)))

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.4f' % self.dataclose[0])
        
        bar = len(self.data)
        self.dt = self.data.datetime.date()
            
        print (self.stoch.lines.percK[0])
        print (self.stoch.lines.percD[0])
        
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            
           

            # Not yet ... we MIGHT BUY if ...
            if self.stoch.lines.percK[-1] < self.stoch.lines.percD[-1] and self.stoch.lines.percK[0] > self.stoch.lines.percD[0]:

                self.cash_before = self.broker.getcash()
                value = self.broker.get_value()
                entry = self.buy(size=self.p.size)
                entry.addinfo(name='Entry')
                
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:
            
            '''
            First check if margin close out conditions are met. If not, check
            to see if we should close the position through the strategy rules.

            If a close out occurs, we need to addinfo to the order so that we
            can log it properly later
            '''
            mco_result = self.check_mco(value=self.broker.get_value(),margin_used=self.margin)

            if mco_result == True:
                close = self.close()
                close.addinfo(name='MCO')

            elif bar in self.closebars:
                close = self.close()
                close.addinfo(name='Close')
                

            if self.stoch.lines.percK[-1] > self.stoch.lines.percD[-1] and self.stoch.lines.percK[0] < self.stoch.lines.percD[0]:

                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
                
        
    def check_mco(self, value, margin_used):
        '''
        Make a check to see if the margin close out value has fallen below half
        of the margin used, close the position.

        value: value of the portfolio for a given data. This is essentially the
        same as the margin close out value which is balance + pnl
        margin_used: Initial margin
        '''

        if value < (margin_used /2):
            return True
        else:
            return False


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    datapath = os.path.join(modpath, 'backtrader\datas\EURUSD1h_01-01_01-04.txt')

    # Create a Data Feed
    data = btfeed.GenericCSVData(
        dataname=datapath,

        fromdate=datetime.datetime(2019, 1, 1),
        todate=datetime.datetime(2019, 4, 1),

        nullvalue=0.0,

        dtformat=('%Y%m%d'),
        tmformat=('%H:%M:%S'),

        datetime=0,
        timestamp=1,
        open=2,
        high=3,
        low=4,
        close=5,
        volume=6
    )

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)
    
    #Variable for our starting cash
    startcash = 10000

    # Set our desired cash start
    cerebro.broker.setcash(startcash)

    # Add a FixedSize sizer according to the stake
#     cerebro.addsizer(bt.sizers.FixedSize, stake=10)

    # Set the commission
    cerebro.broker.setcommission(leverage=50)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything

    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
    
    results = cerebro.run()
#     strat = results[0]
#     pyfoliozer = strat.analyzers.getbyname('pyfolio')
#     returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
    
# #     pyfolio showtime
    
#     pf.create_full_tear_sheet(
#         returns,
#         positions=positions,
#         transactions=transactions,
# #         gross_lev=gross_lev,
#         live_start_date='2019-01-02',  # This date is sample specific
#         round_trips=True)

    
    # At this point tables and chart will show up
    
    #Commission
    cerebro.broker.setcommission(commission=2.0, margin=2000.0, mult=10.0)

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    portvalue = cerebro.broker.getvalue()
    pnl = portvalue - startcash

    print('Final Portfolio Value: ${}'.format(portvalue))
    print('P/L: ${}'.format(pnl))

    #Finally plot the end results
    cerebro.plot(style='candlestick')    
    

    
    
    
    

Starting Portfolio Value: 10000.00


ValueError: unconverted data remains:  22:00:00